In [1]:
using TSSOS
using DynamicPolynomials
using CSV
using DataFrames
using Statistics

In [2]:
dis = CSV.read("distanceSet.csv", header=1);
taxi=[1.38,0.77,2.52,1.15,1.2,3.53,1.0,2.2,0.95,1.42,1.7,2.77,0.57, 0.81,21.47, 1.13,1.19, 0.83, 0.93, 3.3,0.71, 1.04, 11.2, 0.72, 2.63, 0.82, 10.32, 3.51,7.25, 0.48, 2.5, 1.5, 0.4, 1.86, 2.34, 1.42,2.6,1.4, 1.02, 5.96, 0.84, 1.2, 9.3, 1.66,2.0, 2.55, 2.04, 17.87,0.6, 0.57];

In [3]:
# new formulation 3 19/11
# calculate 3 objective: intra-fair inter-fair customer-care

function F3_tssos(icount,jcount,taxi,lam1,lam2,lam3)
    #icount=5 # the number of customers
    #jcount=5 # the number of drivers
    #female=[2,4]
    #male=[1,3,5]
    female=collect(2:2:jcount)
    male=collect(1:2:jcount)
    
    mu1=10
    mu2=10
    mu3=0

    #lam1=1
    #lam2=1
    #lam3=0
    
    dis=rand(icount,jcount)
    
    # set nc operators
    @polyvar u[1:jcount] M[1:icount*jcount]; #z[1:jcount]; 
    var=vcat(u,M);

    #l2=[z[j]-1+(u[j]+0.01)/1.01 for j in 1:jcount];
    #l3=[z[j]+1-(u[j]+0.01)/1.01 for j in 1:jcount];
    #l4=[b1-(1/length(female))*sum(z[j] for j in female),b1-(1/length(male))*sum(z[j] for j in male)];
    #l5=[b2-sum(M[(i-1)*(jcount)+j]*dis[i,j] for j in 1:jcount) for i in 1:icount];
    l6=[1-sum(M[(i-1)*(jcount)+j] for i in 1:icount) for j in 1:jcount];
    l7=[M[(i-1)*(jcount)+j]*(M[(i-1)*(jcount)+j]-1) for i in 1:icount for j in 1:jcount]; #equality #numeq=25

    obj=lam1*sum((1-(u[j]+0.01)/1.01)^2 for j in 1:jcount)+lam2*(sum((u[j]+0.01)/1.01 for j in female)-sum((u[j]+0.01)/1.01 for j in male))^2+lam3*sum(M[(i-1)*(jcount)+j]*dis[i,j] for j in 1:jcount for i in 1:icount)+sum(mu1*(sum(M[(i-1)*(jcount)+j]*(taxi[i]-dis[i,j]) for i in 1:icount)-u[j])^2  for j in 1:jcount)+sum(mu2*(sum(M[(i-1)*(jcount)+j] for j in 1:jcount)-1)^2  for i in 1:icount)

    # pop
    pop=vcat(obj,l6,l7);

    d=1; # the relaxation order
    opt,sol,data=cs_tssos_first(pop,var,d,numeq=icount*jcount,TS="MD",solution=true);
    #opt,sol,data=cs_tssos_higher!(data,TS="MD")
    
    u_sol=sol[1:jcount]
    M_sol=sol[jcount+1:end]
    #intra5=sum((1-(u_sol[j]+0.01)/1.01)^2 for j in 1:jcount)
    mu=sum((u_sol[j]+0.01)/1.01 for j in 1:jcount)/jcount
    intra4=1/(2*mu*jcount^2)*sum(abs((u_sol[k]+0.01)/1.01-(u_sol[j]+0.01)/1.01) for k in 1:jcount for j in 1:jcount)
    inter3=(sum((u_sol[j]+0.01)/1.01 for j in female)-sum((u_sol[j]+0.01)/1.01 for j in male))^2
    care=sum(M_sol[(i-1)*(jcount)+j]*dis[i,j] for j in 1:jcount for i in 1:icount)
    
    return [intra4,inter3,care]

end

F3_tssos (generic function with 1 method)

In [7]:
# for pareto optimal
icount=5 # the number of customers
jcount=5 # the number of drivers
repeat=10
lam1_range=0:0.1:1  
#lam2_range=1:-0.1:0 #step 0.1
df = DataFrame(Intra_Fair4=Float64[],Inter_Fair3=Float64[],Customer_Care=Float64[],lam1=Float64[],lam2=Float64[],lam3=Float64[])

for lam1 in lam1_range
    for lam2 in 0:0.1:(1-lam1)
        for r in 1:repeat
            obj=F3_tssos(icount,jcount,taxi[1:5],lam1,lam2,1-lam1-lam2)
            push!(df, push!(obj,lam1,lam2,1-lam1-lam2))
        end
    end
end

#df[!,:lam2] .= -df[!,:lam1].+1
#df[!,:lam3] .= 0

CSV.write("ParetoOptimal.csv", df,header=true)

------------------------------------------------------
The clique sizes of varibles:
[16, 14, 9, 6]
[1, 2, 5, 5]
------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 296             
  Cones                  : 0               
  Scalar variables       : 31              
  Matrix variables       : 57              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve term

14  3.4e-06  6.8e-08  3.5e-11  1.00e+00   1.027881523e+00   1.027881389e+00   6.8e-08  0.08  
15  2.2e-07  4.4e-09  5.6e-13  1.00e+00   1.027936936e+00   1.027936928e+00   4.3e-09  0.09  
Optimizer terminated. Time: 0.09    

optimum = 1.027936936446568
------------------------------------------------------
The clique sizes of varibles:
[16, 14, 9, 6]
[1, 2, 5, 5]
------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 296             
  Cones                  : 0               
  Scalar variables       : 31              
  Matrix variables       : 57              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator

Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 296             
  Cones                  : 0               
  Scalar variables       : 31              
  Matrix variables       : 57              
  Integer variables      : 0               

Optimizer  - threads                : 4               
Optimizer  - solved problem         : the primal      
Optimizer  - Constraints            : 296
Optimizer  - Cones                  : 1
Optimizer  - Scalar variables       : 32                conic                  : 27              
Optimizer  - Semi-definite variables: 57                scalarized             : 1760            
Factor     - setup time             : 0.02              dense det. time        : 0.00            
Factor     - ML order time          : 0.02  

11  6.7e-04  1.2e-05  8.3e-08  9.98e-01   1.369126519e+00   1.369103028e+00   1.2e-05  0.06  
12  1.5e-04  2.7e-06  8.8e-09  9.98e-01   1.373482016e+00   1.373477292e+00   2.7e-06  0.06  
13  3.7e-05  6.6e-07  1.0e-09  9.99e-01   1.374737605e+00   1.374736491e+00   6.6e-07  0.09  
14  4.1e-06  7.3e-08  3.9e-11  1.00e+00   1.375116638e+00   1.375116504e+00   7.3e-08  0.09  
15  7.2e-08  5.8e-09  8.8e-14  1.00e+00   1.375169643e+00   1.375169641e+00   1.3e-09  0.09  
Optimizer terminated. Time: 0.09    

optimum = 1.3751696433441358
------------------------------------------------------
The clique sizes of varibles:
[16, 14, 9, 6]
[1, 2, 5, 5]
------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 296             
  Cones                  : 0               
  Scalar variables       : 31              
  Mat

------------------------------------------------------
The clique sizes of varibles:
[20, 16, 13, 10]
[1, 2, 1, 5]
------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 366             
  Cones                  : 0               
  Scalar variables       : 31              
  Matrix variables       : 52              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve te

...

11  8.5e-05  1.5e-06  3.7e-09  1.00e+00   1.128129391e+00   1.128126631e+00   1.5e-06  0.11  
12  8.8e-06  1.6e-07  1.2e-10  1.00e+00   1.128617350e+00   1.128617120e+00   1.6e-07  0.14  
13  5.6e-07  1.0e-08  1.9e-12  1.00e+00   1.128662420e+00   1.128662407e+00   1.0e-08  0.14  
14  3.2e-08  7.1e-09  2.2e-14  1.00e+00   1.128663714e+00   1.128663714e+00   5.4e-10  0.16  
Optimizer terminated. Time: 0.16    

optimum = 1.1286637141498692
------------------------------------------------------
The clique sizes of varibles:
[20, 16, 13, 10]
[1, 2, 1, 5]
------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 366             
  Cones                  : 0               
  Scalar variables       : 31              
  Matrix variables       : 52              
  Integer variables      : 0               

Optimize

2   3.2e+01  5.1e-01  6.8e-01  -9.17e-01  -5.035895388e+01  -5.117957210e+01  5.1e-01  0.03  
3   1.2e+01  1.8e-01  3.5e-01  -8.32e-01  -1.860439109e+02  -1.888087849e+02  1.8e-01  0.05  
4   4.2e+00  6.7e-02  5.7e-02  -4.68e-02  -1.275786946e+02  -1.280212716e+02  6.7e-02  0.06  
5   1.3e+00  2.0e-02  5.0e-03  1.66e+00   -2.355869237e+01  -2.356546458e+01  2.0e-02  0.06  
6   2.6e-01  4.1e-03  5.7e-04  1.34e+00   -2.553075845e+00  -2.562820474e+00  4.1e-03  0.08  
7   5.6e-02  8.9e-04  5.5e-05  1.08e+00   7.476817015e-01   7.459571778e-01   8.9e-04  0.08  
8   1.7e-02  2.6e-04  8.9e-06  1.02e+00   9.057375545e-01   9.051838713e-01   2.6e-04  0.08  
9   4.0e-03  6.3e-05  1.0e-06  9.96e-01   9.928706821e-01   9.927341936e-01   6.3e-05  0.08  
10  6.6e-04  1.0e-05  7.1e-08  9.99e-01   1.014927235e+00   1.014903386e+00   1.0e-05  0.09  
11  3.5e-05  5.5e-07  8.7e-10  1.00e+00   1.017934948e+00   1.017933664e+00   5.5e-07  0.09  
12  1.7e-06  2.7e-08  9.3e-12  1.00e+00   1.018095227e+00   

Excessive output truncated after 524312 bytes.

Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 366             
  Cones                  : 0               
  Scalar variables       : 31              
  Matrix variables       : 52              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.02            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.03    
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimizat

"ParetoOptimal.csv"